In [6]:
import spacy

# nlp=spacy.blank("en")
nlp = spacy.load("en_core_web_sm")
text = "West Chestertenfieldville was referenced in Mr. Deeds"

doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

West Chestertenfieldville GPE
Deeds PERSON


In [7]:
ruler = nlp.add_pipe("entity_ruler")
nlp.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'entity_ruler': {'assigns': ['doc.ents', 'token.ent_type', 'token.ent_iob'],
   'requires': [],
   'scores': ['ents_f', 'ent

In [20]:
patterns = [
    {"label": "GPE", "pattern": "West Chestertenfieldville"}
]
ruler.add_patterns(patterns)

doc2=nlp(text)
for ent in doc2.ents:
    print(ent.text, ent.label_)
    # result is numeric, dont know yet why it's not a string like above. Now I know - I didn't add the floor at the end

Poland GPE


In [24]:
nlp3 = spacy.load("en_core_web_sm")
ruler=nlp3.add_pipe("entity_rulers", before="parser")
patterns = [ 
    {"label": "GPE", "pattern": "West Chestertenfieldville"},
    {"label": "FILM", "pattern": "Mr. Deeds"}
]

ruler.add_patterns(patterns)

doc=nlp3(text)

for ent in doc.ents:
    print(ent.text, ent.label_)
    # for some reason this part of the code returns a lot of mistakes I don't understand yet, gonna move on with the lecture

SyntaxError: unexpected EOF while parsing (<ipython-input-24-4b9c4bfba32f>, line 14)

In [ ]:
# toponym resolution
# Paris - France, Kentucky or Texas - it's always a challenge

In [ ]:
# Another example

nlp = spacy.load("en_core_web_sm")
text = "The village of Treblinka is in Poland. Treblinka was also an extermination camp."
doc = nlp(text)
#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

# taking a look at the result, model hasn't encountered a world Treblinka before

Poland GPE


In [ ]:
# MATCHERS

In [25]:
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
pattern = [{"LIKE_EMAIL": True}]
matcher.add("EMAIL_ADDRESS", [pattern])
doc = nlp("This is an email address: wmattingly@aol.com")
matches = matcher(doc)
print(matches)

[(16571425990740197027, 6, 7)]


In [26]:
# tutor mentioned a term lexeme here. 
print(nlp.vocab[matches[0][0]].text)

EMAIL_ADDRESS


In [5]:
with open ("data/wikimlk.txt", "r") as f:
    text=f.read()
# print(text)

nlp=spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN"}]
matcher.add("PROPER_NOUNS", [pattern])
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])


112
(3232560085755078826, 0, 1) Martin
(3232560085755078826, 1, 2) Luther
(3232560085755078826, 2, 3) King
(3232560085755078826, 3, 4) Jr.
(3232560085755078826, 6, 7) Michael
(3232560085755078826, 7, 8) King
(3232560085755078826, 8, 9) Jr.
(3232560085755078826, 10, 11) January
(3232560085755078826, 15, 16) April
(3232560085755078826, 23, 24) Baptist


In [14]:
# IMPROVING
import spacy
from spacy.matcher import Matcher
nlp=spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}, {"POS": "VERB"}]
matcher.add("PROPER_NOUNS", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key=lambda x:x[1])
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])


5
(3232560085755078826, 69, 71) King advanced
(3232560085755078826, 117, 119) King participated
(3232560085755078826, 311, 316) Director J. Edgar Hoover considered
(3232560085755078826, 384, 386) King won
(3232560085755078826, 525, 528) United States beginning


In [20]:
import json
with open("data/Alice-in-Wonderland.json", "r") as f:
    data=json.load(f)

text=data
print(text)

JSONDecodeError: Invalid \escape: line 1 column 2264 (char 2263)

In [ ]:
# CUSTOM COMPONENTS

import spacy

nlp=spacy.load("en_core_web_sm")
doc=nlp("Britain is a place. Mary is a doctor")
for ent in doc.ents:
    print(ent.text, ent.label_)

Britain GPE


In [ ]:
from spacy.language import Language
@Language.component("remove_gpe")
def remove_gpe(doc):
    original_ents=list(doc.ents)
    for ent in doc.ents:
        if ent.label_=="GPE":
            original_ents.remove(ent)
    doc.ents=original_ents
    return (doc)

In [ ]:
nlp.add_pipe("remove_gpe")


<function __main__.remove_gpe(doc)>